### Frequência

In [38]:
import pandas as pd

df_frequencia = pd.read_excel('frequencia.xlsx')
df_funcionarios = pd.read_excel('funcionarios.xlsx')
df_turnos = pd.read_excel('turnos.xlsx')


In [39]:
# Ajuste do formato da 'Data'
df_frequencia['Data'] = pd.to_datetime(df_frequencia['Data'], unit='ms').dt.strftime('%d-%m-%Y')

In [40]:
print(df_frequencia.isnull())

      Id_funcionario   Data  Entrada  Saida  Faltou  Horas_extras
0              False  False    False  False    True         False
1              False  False    False  False    True         False
2              False  False    False  False    True         False
3              False  False     True   True    True         False
4              False  False    False  False    True         False
...              ...    ...      ...    ...     ...           ...
2995           False  False    False   True    True         False
2996           False  False    False  False    True         False
2997           False  False     True   True    True         False
2998           False  False    False  False    True         False
2999           False  False    False  False    True         False

[3000 rows x 6 columns]


In [41]:
# Substituindo valores nulos por "Faltas" e 99:99 por "Inválido"
df_frequencia['Entrada'] = df_frequencia['Entrada'].fillna("").astype(str).str.strip()
df_frequencia['Saida'] = df_frequencia['Saida'].fillna("").astype(str).str.strip()

cond_sim = (df_frequencia['Entrada'] == "") & (df_frequencia['Saida'] == "")
df_frequencia.loc[cond_sim, 'Faltou'] = "Sim"

df_frequencia.loc[df_frequencia['Entrada'] == "99:99", 'Faltou'] = "Inválido"

df_frequencia['Faltou'] = df_frequencia['Faltou'].fillna("Não")

C:\Users\53144094889\AppData\Local\Temp\ipykernel_16492\998549533.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Sim' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_frequencia.loc[cond_sim, 'Faltou'] = "Sim"


In [47]:
df_frequencia = pd.read_excel('frequencia_limpo.xlsx')

### Funcionários

In [48]:
# Eliminando valores "99:99"
df_frequencia = df_frequencia[df_frequencia['Entrada'] != "99:99"]

# Groupby para juntar horas por funcionário
df_funcionarios["Horas_extras"] = df_funcionarios["Id_funcionario"].map(
    df_frequencia.groupby("Id_funcionario")["Horas_extras"].sum()
)

In [49]:
df_frequencia.head(10)

,Id_funcionario,Data,Entrada,Saida,Faltou,Horas_extras
0,1,07-05-2025,11:38:44,12:03:04,Não,0.1
1,1,06-05-2025,11:42:04,09:58:54,Não,2.2
2,1,05-05-2025,15:10:54,01:22:06,Não,0.3
3,1,04-05-2025,NaN,NaN,Sim,0.0
4,1,03-05-2025,07:02:53,08:13:02,Não,0.1
5,1,02-05-2025,07:41:48,08:15:59,Não,1.6
7,1,30-04-2025,21:39:10,21:48:17,Não,2.1
8,1,29-04-2025,18:08:57,18:29:27,Não,2.9
9,1,28-04-2025,NaN,NaN,Sim,0.0
10,1,27-04-2025,23:52:45,12:16:17,Não,1.8


In [58]:
df_funcionarios.to_excel('funcionarios_limpo.xlsx', index=False) 

In [51]:
# Groupby para juntar faltas por funcionário

df_funcionarios["Faltou"] = df_funcionarios["Id_funcionario"].map(
    df_frequencia[df_frequencia["Faltou"] == "Sim"]
    .groupby("Id_funcionario")["Faltou"]
    .count()
)


In [56]:
# Transformando faltas em int
df_funcionarios['Faltou'] = pd.to_numeric(df_funcionarios['Faltou'], errors='coerce').astype('Int64')

,Id_funcionario,Nome,Sexo,Data_admissao,Cargo,Setor,Turno,Salario,Horas_extras,Faltou
0,1,Dr. Kaique da Cruz,Não Informado,1555027200000,Analista,Produção,Noite,11421.21,35.0,3
1,2,Let,F,1730160000000,Auxiliar,Comercial,Manhã,11081.07,30.5,4
2,3,Sophia Cassiano,M,1634083200000,Analista,Logística,Tarde,5034.62,48.3,2
3,4,Ravy Rodrigues,F,1517011200000,Operador,RH,Manhã,11270.30,41.8,1
4,5,Maria Sophia Almeida,F,1532131200000,Auxiliar,Produção,Noite,4345.00,39.0,3
5,6,Alice Vargas,F,1628640000000,Gerente,Financeiro,Tarde,6102.74,39.4,1
6,7,Isis Porto,F,1500595200000,Analista,Financeiro,Tarde,6436.49,36.5,3
7,8,José Pedro Moraes,M,1684713600000,Auxiliar,RH,Tarde,4894.82,35.0,5
8,9,Dra. Melina Nogueira,F,1595808000000,Supervisor,Financeiro,Tarde,3126.90,44.0,2
9,10,Mathias Porto,F,1700265600000,Analista,Comercial,Tarde,24545.50,41.6,1
